# Q-Learning: Advanced-Nim
Hausarbeit Computational Intelligence - Michael Hopp

Fachhochschule Erfurt - Wintersemester 2022/2023

Prüfer: Prof. Dr. Frank Michael Dittes

## Aufgabenstellung
Der Q-Learning-Algorithmus soll auf das Finden der optimalen Strategie für ein "einfaches" Spiel (hier Advanced-Nim) übertragen werden. Es soll möglich sein, die Q-Table einzusehen und interaktiv gegen einen Computergegner (AI) zu spielen.

Als Zusatzaufgabe soll es möglich sein, die Konfiguration der Stapelanzahl und deren Stapelgrößen flexibel selbst zu bestimmen.

## Einleitung
Unter dem Begriff "Nim-Spiel" (engl. "Nim-Game") werden verschiedene Varianten von Spielen verstanden, bei denen durch **Zwei Spieler** abwechselnd aus einer Sammlung gleicher Gegenstände eine - je nach Regelwerk eingeschränkte - Anzahl von Gegenständen **aufgenommen**, und somit vom *Spielfeld* entfernt werden. Ziel der Spieler ist es, die verbleibende Anzahl der Gegenstände so zu manipulieren, dass der Gegenspieler keine andere Wahl hat, als den letzten verbleibenden Gegenstand aufzunehmen (Misére-Variante) oder selbst den letzten Gegenstand zu nehmen (Standardvariante).

Diese Spiele unterliegen mathematischen Regeln, die optimale Lösungsstrategien ableiten lassen und zur Erstellung von Computer-Gegnern verwendet werden können (vgl. [Nim-Computer "Nimatron" 1940 und "Nimrod" 1951 für das 4-Reihen Spiel mit Gewinnstrategie nach Bouton](https://de.wikipedia.org/wiki/Nim-Spiel)).

Der Q-Learning-Algorithmus soll dazu dienen, eine solche optimierte Lösungsstrategie zu finden. Dafür müssen zuerst die eigentlichen Spielregeln verstanden werden und eine angemessene Zustandskodierung gewählt werden, bevor das Programm dann implementiert wird. Die hier entstehende Ergebnisstrategie soll zum Vergleich gegen eine etablierte Gewinnstrategie antreten und das Verhalten der AI analysiert werden.

## Spielregeln
Gespielt wird [Advanced Nim](https://www.youtube.com/watch?v=vt0ZddT_kBQ) wie es vom *Youtube Kanal Scam Nation* vorgestellt wird.

Es spielen zwei Spieler gegeneinander. Vorzugsweise werden als Gegenstände Streichhölzer oder etwas ähnliches verwendet.

Die Streichhölzer werden in insgesamt drei Stapeln mit drei, fünf und sieben Streichhölzern ausgelegt (damit ist die Anzahl der Streichhölzer für diese Spielvariante immer 15).

**Spielaufbau:**
``` 
3 III
5 IIIII
7 IIIIIII
```
Ein Spieler kann in seinem Zug **beliebig viele** Streichhölzer **aus einer Reihe** nehmen. 

Verloren hat der Spieler, der das letzte Streichholz aufnimmt (entspricht Misére-Variante, die durch Schadenfreude vermeintlich mehr Spaß macht).

### Gewinnstrategie nach Scam Nation
Wie könnte ein Mensch vorgehen, um zu gewinnen?
Es existieren sogenannte **Pole Positions** (Schlüsselpositionen), die durch clevere Züge zu erzeugen sind, um dem Gegenspieler keine Möglichkeit auf einen Sieg zu lassen. Dadurch ist es nicht nötig, alle möglichen Folgezüge vorauszusehen (wie z.B. beim Schach), sondern die Strategie führt den Spieler von einer *sicheren Position* in die nächste.

**Pole Positions** (Physische Ordnung irrelevant):
```
1 1 1
1 2 3
2 4 6
1 4 5
2 gleiche Stapel
```

Zur Verfolgung dieser Gewinnstrategie wird dem Gegenspieler der erste Zug überlassen. Anschließend wird versucht, als Gegenzug immer eine dieser Pole Positions zu erzeugen. Sobald nur noch wenige Streichhölzer liegen, gilt es, besonders aufmerksam zu sein, um den Siegeszug nicht zu verpassen oder durch die Anwendung der Strategie sogar zu verbauen (Beispiel: 1 1 0 als "2 gleiche Stapel" zu erzeugen, führt im Gegenzug zu 1 0 0 und damit zur eigenen Niederlage).

**Schwachstelle:** Als **Konter** zu dieser Strategie kann der Gegenspieler im ersten Zug nur ein einziges Streichholz von einem beliebigen Stapel entnehmen, wodurch im zweiten Zug keine Pole Position erzeugt werden kann. Nach dem zweiten Zug kann somit der erste Spieler die Oberhand gewinnen und selbst eine Pole Position erzeugen, aus der sich der zweite Spieler nicht befreien kann - solange bei der Erzeugung der Positionen oder dem abschließenden Zug kein Fehler begangen wird.

## Q-Learning Konzept
Beim Q-Learning, welches zur Klasse des Reinforcement Learning gehört, wird die Qualität einer Aktion in einem Zustand ermittelt. Dabei muss lediglich der Wert (Reward) des Endzustandes festgelegt werden (*Gewonnen* ist gut und gibt 100 "Punkte", während *Verloren* schlecht ist und -100 "Punkte" gibt). Anhand dieses Endergebnisses wird beim Durchlaufen von möglichen Zustandsketten (festgelegt durch die "Spielregeln") allmählich die Q-Table mit weiteren Werten gefüllt. Die Q-Table gibt für jede mögliche Aktion jedes Zustandes einen Qualitätswert (Q-Value) an. Da zuerst nur der Endzustand bewertet ist, füllt sie sich "von hinten". Das ist vergleichbar mit einer menschlichen Strategie zum Lösen von Labyrinthen (die sich übrigens ebenfalls für Q-Learning anbieten): Dabei wird nicht etwa vom Start des Labyrinths ein Weg zum Ausgang gesucht, sondern umgekehrt vom Ausgang der Weg zum Start "zurückverfolgt".

### Q-Value und Discount-Faktor
Der Qualitätswert eines Zustandes wird aus dem sofortigen Reward, sowie dem im nächsten Zustand möglichen Reward berechnet. Anhand eines Discount-Faktors kann bestimmt werden, wie viel Wert auf zukünftige Rewards gelegt werden soll. Eine Analogie dazu: Ein Kind hat 2€ (Zustand) und könnte sich von diesem Geld ein Eis kaufen (Aktion). Der Reward wäre an dieser Stelle das Eis und da das Geld verbraucht ist, könnte nichts Weiteres gekauft werden. Nun könnte das Kind seine 2€ auch gewinnbringend investieren (Aktion). Diese Aktion würde keinen sofortigen Reward liefern. Allerdings besteht die Möglichkeit, dass das Kind einen Zustand erreicht, in dem es mehr als seine vorherigen 2€, zum Beispiel nämlich ganze 4€ zurückerlangt. Von diesem Zustand aus könnte sich das Kind *zwei* Portionen Eis kaufen (Aktion) und dementsprechend einen höheren Reward als zuvor erzielen. Welche Qualität die Aktionen haben und für welche sich das Kind schließlich entscheidet, hängt von dem Discount-Faktor ab, mit dem es die aufgeschobene Belohnung bewertet. Ist der Dicount-Faktor hoch, so fließen zukünftige Rewards stärker ein.

Ein ähnliches Beispiel für den Effekt von Belohnungsaufschub und die Bereitschaft dazu liefert das [Stanford Marshmallow Experiment](https://en.wikipedia.org/wiki/Stanford_marshmallow_experiment).

### Exploration vs Exploitation
Welche Aktion ist beim Lernen die *richtige*? Dafür gibt es verschiedene Strategien, die sich aus dem Verhältnis von Exploration zu Exploitation ergeben. Beim Durchlaufen der Zustandsketten könnten immer zufällige Aktionen oder stets die Aktion mit dem besten Q-Value gewählt werden:
- Exploration: Die reine Zufallsvariante ist nicht sonderlich zielstrebig, dafür werden bei genügend häufiger Wiederholung alle Zustandsketten durchlaufen. Wenn die Anzahl der Zustände und der Aktionen hoch ist, muss die Anzahl der Wiederholungen entsprechend umfangreich sein. 
- Exploitation: Bei der Variante, die immer nur die bisher beste Aktion wählt, kann es dazu kommen, dass immer die gleichen Zustände durchlaufen werden, wodurch ggfs. bessere Aktionen nie probiert werden. Diese Variante findet keinen Weg aus einem lokalen Optimum heraus, falls einmal eines erreicht wird.

Besser ist es, beide Verfahren zu kombinieren:
- Prinzipiell kann die Aktion gewählt werden, die den besten Q-Value liefert, doch um die Chance zu haben, lokale Optima zu vermeiden, wird in sinnvollem Maße ein Zufallselement eingesetzt. Beispiel: In 10% der Fälle soll statt der besten Aktion eine beliebige (zulässige) Aktion gewählt werden.

Ein Beispiel für Exploration und Exploitation aus dem Alltag wäre die Abwägung, in welchem Restaurant man essen gehen möchte: 
- Exploitation: Geht man zum Altbewährten, von dem man sicher weiß, wie gut es ist? 
- Exploration: Oder probiert man ein neues Lokal aus, das die Chance bietet, das neue Lieblingsrestaurant zu werden, sich jedoch auch als Reinfall herausstellen könnte?

## Implementierung
Wie könnte ein Programm aussehen, das einer AI diese Variante von Nim beibringt?
Essentiell ist dabei eine sinnvolle Zustandskodierung. Außerdem muss die Implementierung genügend Flexibilität bieten, um variable Stapelmengen und -größen zu ermöglichen.

### Vorüberlegungen
Der wohl wichtigste Baustein beim Q-Learning-Algorithmus ist die Zustandskodierung. Neben den Spielregeln (Spielende und erlaubte Züge) muss dem Computer der aktuelle Spielzustand vermittelt werden, sodass er darauf das Q-Learning anwenden kann. Bevor also drauflosprogrammiert wird, sind die Dimensionen des Spiels zu erfassen und Möglichkeiten für die Zustandskodierung zu betrachten und abzuwägen.

Die Anzahl der möglichen Zustände ergibt sich aus dem *Produkt der Stapelgrößen+1*. Beispiel für das Standardspiel [3 5 7]: ((3+1) * (5+1) * (7+1) = **192**)

In jedem Zustand entspricht die Anzahl der möglichen Aktionen der *Summe der Stapelgrößen* (Summe der Hölzchen). Mit jedem Zug sinkt die Anzahl der möglichen Aktionen demnach um die entnommene Anzahl der Gegenstände.
- Beispiel für Zustand [3 5 7]: 3+5+7 = 15
- Beispiel für Zustand [0 0 2]: 0+0+2 = 2 (entweder zwei nehmen und verlieren oder nur eins nehmen und gewinnen)

**Anforderung:** Erweiterbarkeit, um variable Stapelanzahl und beliebige Menge von Streichhölzern pro Stapel zu ermöglichen

**Wunsch:** Die Konfiguration der Stapel und deren Hölzchenzahl (Stapelgröße) soll möglichst simpel für den Benutzer sein

**Idee:** Zustände als Arrays aus Stapelgrößen. Stapelanzahl ergibt sich aus Indizes und eine Aktion umfasst einen Stapelindex, sowie die zu subtrahierende Anzahl von Hölzchen in diesem Array-Feld (Stapel). Mögliche Aktionen wären je Stapel eins bis maximal die Größe des Stapels (höchstens bis das Feld null ist).

Zur Reduzierung der Zustands-Dubletten (z.B. 0,1 = 1,0), könnte das Array der Stapelgrößen nach jeder Anpassung sortiert werden.

Alternativ könnten Zustände als Unsigned Integer mit den Stapeln über 10er-Potenzen kodiert werden. Beispiel: 357 als Startkonfiguration. Dadurch würden zur Dekodierung Mechanismen wie der Modulo-Operator benötigt werden. Der Vorteil wäre eine geringere Speicherbelastung. Die Sortierung zum Einsparen von Zustands-Dubletten würde jedoch umständlicher werden. Außerdem wären die Stapel somit auf maximal neun Objekte limitiert (single digits).

Die Zustände könnten auch durchnummeriert werden, wobei die Kodierung (mit Überspringen von Zustands-Dubletten) nicht mehr einfach menschenlesbar ausfällt. Ausschnitt der Ganzzahlkodierung mit Durchnummerierung für das Standardspiel [3 5 7]:
```
State Code
0,0,0 0
0,0,1 1
0,0,2 2
...
0,0,7 7
0,1,1 8
0,1,2 9
...
0,1,7 14
0,2,3
0,2,4
...
0,2,7
...
0,5,6
0,5,7
1,1,1
1,1,2
...
1,1,7
1,2,2
...
1,2,7
1,3,3
1,3,7
...
1,5,5
1,5,7
2,2,2
...
2,2,7
...
```


### Zustandskodierung und Q-Table Datenstruktur
Als Ergebnis der Vorüberlegung wird zugunsten der Flexibilität realitätsgetreu folgende Zustandskodierung und Q-Table Datenstruktur eingesetzt.

Die **Q-Table Datenstruktur** ist ein Dictionary bestehend aus den aufgelisteten Paaren von Zustand+Aktion und deren Q-Value. Die **Zustände** sind wiederum Arrays mit den Stapelgrößen. **Aktionen** (Zustandsüberführungen) sind Tupel mit Stapelindex und Anzahl der zu entnehmenden Gegenstände. 

**Beispiele:**
```
Level 1 Beispiel:
Q = {
    (state,action) : Q-value
    ...
    }
```

``` 
Level 2 Beispiel:

Q = {
    ([3,5,7], (0,1)) : 0
    ...
    }
```

### Programm
Das Programm ist in Python geschrieben und erlaubt, mit variabler Stapelanzahl und -größe zu spielen.
Demnach startet das Programm mit der Abfrage der Spielfeldkonfiguration, auf der die AI dann trainiert wird. Gespielt wird im Modus Mensch gegen AI.

#### Konfiguration des Spielfelds
Hier werden die Stapelgrößen festgelegt, um das Ausgangsspielfeld zu bestimmen, auf dem die AI trainiert werden soll. Die Stapel werden sortiert und die Anzahl der möglichen Zustände dieser Konfiguration berechnet und ausgegeben.

In [12]:
# Konfiguriere das Spielfeld/Board
import numpy as np
import copy


def configure_game():
    #validate input
    isnumeric = False
    while not isnumeric:
        isnumeric = True #Do-While simulation
        
        pileQuantities = input("Geben Sie die gewünschte Stapelgröße je Stapel (per Leerzeichen getrennt) ein (Beispiel: 3 5 7): ")

        for x in pileQuantities.strip().split():
            if not x.isnumeric():
                isnumeric = False
                print(str(x) + " ist keine gültige Eingabe. Bitte nur Zahlen eingeben")
                break
    
    board = [int(x) for x in pileQuantities.split()]
    print("Ihre unsortierte Eingabe: " + str(board))
    board = np.sort(board)
    print("Sortiertes Spielfeld: " + str(board))
    return board
        
    
def draw_board(state):
    print("--------------------------------------------")
    print("Spielfeld:")
    for row in range(len(state)):
        out = "Stapel " + str(row) + ": " + str(state[row]) +  "\t"
        for i in range(state[row]):
            out += " I"
        print(out)

    
startboard = configure_game()
draw_board(startboard)

num_States = 1
for pile in range(len(startboard)):
    num_States *= startboard[pile]+1
print("Anzahl möglicher Zustände: " + str(num_States))

Geben Sie die gewünschte Hölzchenanzahl pro Stapel (per Leerzeichen getrennt) ein (Beispiel: 3 5 7):  3 5 7


Ihre unsortierte Eingabe: [3, 5, 7]
Sortiertes Spielfeld: [3 5 7]
--------------------------------------------
Spielfeld:
Stapel 0: 3	 I I I
Stapel 1: 5	 I I I I I
Stapel 2: 7	 I I I I I I I
Mögliche Zustände: 192


#### Definition der Klassen und Methoden für Nim und die AI, sowie das Trainieren und Spielen

In [2]:
import math
import random
import time
import numpy as np


# Spiel-Klasse
class Nim():
    
    # Initialisiere Spielfeld (board)
    def __init__(self, startboard=[3, 5, 7]):
        self.piles = startboard.copy()
        self.player = 0    # bestimmt, wer dran ist (0 or 1)
        self.winner = None

    # Gibt alle möglichen Aktionen für die übergebene Stapelkonfiguration als Tupel aus Stapelindex und zu-entnehmender-Anzahl zurück
    @classmethod
    def available_actions(cls, piles):
        actions = set()
        for i, pile in enumerate(piles):
            for j in range(1, pile + 1):
                actions.add((i, j))
        return actions

    # Gibt den aktuell nicht aktiven Spieler aus
    @classmethod
    def other_player(cls, player):
        return 0 if player == 1 else 1

    # Wer dran wechselt auf inaktiven Spieler
    def switch_player(self):
        self.player = Nim.other_player(self.player)

    # Führt eine Aktion (Stapel, Menge) auf dem aktuellen Spielfeld durch
    def move(self, action):
        pile, count = action

        # Validieren
        if self.winner is not None:
            raise Exception("Spiel ist bereits gewonnen")
        elif pile < 0 or pile >= len(self.piles):
            raise Exception("Ungültiger Stapelindex")
        elif count < 1 or count > self.piles[pile]:
            raise Exception("Ungültige Menge im Stapel")

        # Update pile
        self.piles[pile] -= count
        
        # Zug beendet, anderer Spieler ist dran
        self.switch_player()
        
        # Prüfe Spielende. Sieger ist gewechselter Spieler, wegen misère Variante
        if all(pile == 0 for pile in self.piles):
            self.winner = self.player
        
        
# Lerner bzw. Computergegner
class NimAI():

    # Initialisiere AI mit leerem Q-Table
    # Gib Lernrate, Discountfaktor und Epsilonrate als Lernparameter vor
    # Das Q-Learning Dictionary mapt (state, action) Paare zu Q-Value
    # state ist ein Tupel mit Stapelkonfiguration (3, 5, 7)
    # action ist ein Tupel für (Stapelindex, Anzahl)
    def __init__(self, learningrate=0.1, discount=0.8, epsilon=0.1):
        self.q = dict()
        self.learningrate = learningrate
        self.discount = discount
        self.epsilon = epsilon

    # Aktualisiert Q-Table mithilfe von altem Zustand, der daraus verwendeten Aktion und dem
    # resultierenden Zustand, sowie dessen Reward
    def update(self, old_state, action, new_state, reward):
        old_q = self.get_q_value(old_state, action)
        best_future = self.best_future_reward(new_state)
        self.update_q_value(old_state, action, old_q, reward, best_future)

    # Gibt Q-Value für den Zustand und die Aktion aus, falls vorhanden, sonst 0
    def get_q_value(self, state, action):
        # Bedenke, dass der Zustand eine Liste ist!
        if not self.q:  # empty dict
            return 0

        Q = 0
        if (tuple(state), action) in self.q.keys():
            Q = self.q[(tuple(state), action)]
        return Q
    
    # Q-Matrix updaten mit Formel:
    # Q[state, action] = old_q + learningrate * deltaQ
    # Wobei deltaQ = reward + discount * best_future_reward - old_q
    def update_q_value(self, state, action, old_q, reward, best_future_reward):
        deltaQ = reward + self.discount * best_future_reward - old_q
        Q = old_q +  self.learningrate * deltaQ
        self.q[(tuple(state), action)] = Q

    # Gibt für gegebenen Zustand den Q-Value der besten Aktion aus
    def best_future_reward(self, state):
        #print("\tbest_future_reward calls actions_dict mit state = " + str(state))
        actions = self.actions_dict(state)
        if not actions:
            return 0
        return max(actions.values())

    # Gibt für einen Zustand die zu wählende Aktion aus
    # Während Training wird Epsilon-Exploration verwendet
    # Beim Spielen soll die AI den besten Zug machen
    def choose_action(self, state, epsilon=True):
        #print("\tchoose_action calls actions_dict mit state = " + str(state))
        actions = self.actions_dict(state)  # DICT {action:Q}
        if not epsilon:
            return max(actions, key=actions.get)
        else:         
            # Epsilon-Variante (Explore-Exploit)
            p = np.random.random()
            if p < self.epsilon:
                #print("Epsilon obsiegte mit p = " + str(p))
                return list(actions.keys())[np.random.randint(len(actions.keys()))]
            else:
                return max(actions, key=actions.get)
            
            # Komplett random Variante (Exploration pur)
            #return list(actions.keys())[np.random.randint(len(actions.keys()))]

    # Gibt mögliche Aktionen als Dictionary aus
    def actions_dict(self, state):
        actions_set = Nim.available_actions(state)

        actions = dict()  # dict[action]=Q
        
        for action in actions_set:
            actions[action] = self.get_q_value(state, action)
            
        return actions


# Trainiere die AI mit n Spielen gegen sich selbst
def train(n, startboard=[3,5,7]):

    player = NimAI()

    # Spiele n Spiele
    for i in range(n):
        if i % (n/10) == 0:
            print(f"{int(i/n*100)}%:\tSpiele Trainingsspiel {i}")
        game = Nim(startboard)

        # Merke die letzten Züge beider Spieler
        last = {
            0: {"state": None, "action": None},
            1: {"state": None, "action": None}
        }

        # Game loop
        while True:
            # Speichere aktuellen Zustand und Aktion
            state = game.piles.copy()
            action = player.choose_action(game.piles)

            # Merke die letzten Züge beider Spieler
            last[game.player]["state"] = state
            last[game.player]["action"] = action

            # Führe Aktion aus
            game.move(action)
            new_state = game.piles.copy()

            # When game is over, update Q values with rewards
            # Wenn das Spiel vorbei ist, aktualisiere Q-Values
            # Setze Rewards hier
            if game.winner is not None:
                player.update(state, action, new_state, -100)
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    100
                )
                break

            # Keine neuen Rewards, solange das Spiel noch läuft
            elif last[game.player]["state"] is not None:
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    0
                )

    print("100%\tTraining abgeschlossen")

    # Gib trainierte AI aus
    return player


# Spiel interaktiv Mensch gegen AI
# human_player kann 0 oder 1 gesetzt werden, um Zugreihenfolge festzulegen, sonst zufällig
def play(ai, startboard=[3,5,7], human_player=None):

    # Ohne Reihenfolge: Zufälliger Start
    if human_player is None:
        human_player = random.randint(0, 1)

    # Erstelle neues Spiel
    game = Nim(startboard)

    # Game loop
    while True:

        # Zeichne Spielfeld
        print()
        print("Spielfeld:")
        for i, pile in enumerate(game.piles):
            print(f"Stapel {i}: {pile}")
        print()

        # Berechne mögliche Aktionen für Input-Validierung
        available_actions = Nim.available_actions(game.piles)
        time.sleep(1)
        
        # Zeige Aktionen mit Bewertungen
        print("Q-Values")
        for action in sorted(ai.actions_dict(game.piles)) :  # TODO sortiere Liste, für bessere Lesbarkeit
            print("\tAktion: " + str(action) + " Bewertung: " + str(round(ai.get_q_value(game.piles,action),2)))
        # Alternativ (ohne Umbrüche aber direkter)
        #print(str(ai.actions_dict(game.piles)))

        # Mensch am Zug
        if game.player == human_player:
            print("Dein Zug")
            while True:
                pile = int(input("Wähle Stapel: "))
                count = int(input("Wähle Anzahl: "))
                if (pile, count) in available_actions:
                    break
                print("Ungültige Aktion, versuch es nochmal.")

        # Computer am Zug
        else:
            print("AI's Zug")
            pile, count = ai.choose_action(game.piles, epsilon=False) # epsilon false, damit bester Zug gewählt wird
            print(f"AI nahm {count} von Stapel {pile}.")

        # Make move
        game.move((pile, count))

        # Check for winner
        if game.winner is not None:
            print()
            print("GAME OVER")
            winner = "Mensch" if game.winner == human_player else "AI"
            print(f"{winner} hat gewonnen")
            return


#### Training der AI
Trainiert die AI in so vielen Epochen, wie angegeben wird und gibt dann die gesamte Q-Table aus (Ausgabe ggfs. entkommentieren).

In [3]:
# Trainiere das Modell 
ai = train(100000, startboard)

# Zeichne Q-Table
# for entry in sorted(ai.q, reverse=True):
#     print("(State,Action): " + str(entry) + "\tBewertung: " + str(round(ai.get_q_value(entry[0],entry[1]),2)))

0%:	Spiele Trainingsspiel 0
10%:	Spiele Trainingsspiel 10000
20%:	Spiele Trainingsspiel 20000
30%:	Spiele Trainingsspiel 30000
40%:	Spiele Trainingsspiel 40000
50%:	Spiele Trainingsspiel 50000
60%:	Spiele Trainingsspiel 60000
70%:	Spiele Trainingsspiel 70000
80%:	Spiele Trainingsspiel 80000
90%:	Spiele Trainingsspiel 90000
100%	Training abgeschlossen


#### Spielen
Startet das Spiel gegen den Computer. Durch optionale Angabe von 0 beginnt der Mensch, bei 1 der Computer. Ohne diese Festlegung wird der Startspieler zufällig bestimmt.

In [5]:
#Spiele gegen den Computer
play(ai, startboard, 0)


Spielfeld:
Stapel 0: 1
Stapel 1: 2
Stapel 2: 3
Stapel 3: 4
Stapel 4: 5

Q-Values
	Aktion: (0, 1) Bewertung: 31.15
	Aktion: (1, 1) Bewertung: -1.47
	Aktion: (1, 2) Bewertung: -9.57
	Aktion: (2, 1) Bewertung: 27.63
	Aktion: (2, 2) Bewertung: -5.39
	Aktion: (2, 3) Bewertung: 3.25
	Aktion: (3, 1) Bewertung: -5.1
	Aktion: (3, 2) Bewertung: -15.32
	Aktion: (3, 3) Bewertung: -13.65
	Aktion: (3, 4) Bewertung: -26.05
	Aktion: (4, 1) Bewertung: 27.81
	Aktion: (4, 2) Bewertung: -2.83
	Aktion: (4, 3) Bewertung: -15.89
	Aktion: (4, 4) Bewertung: -22.85
	Aktion: (4, 5) Bewertung: -19.52
Dein Zug


Wähle Stapel:  0
Wähle Anzahl:  1



Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 3
Stapel 3: 4
Stapel 4: 5

Q-Values
	Aktion: (1, 1) Bewertung: -12.24
	Aktion: (1, 2) Bewertung: -12.84
	Aktion: (2, 1) Bewertung: -7.36
	Aktion: (2, 2) Bewertung: -10.39
	Aktion: (2, 3) Bewertung: -10.92
	Aktion: (3, 1) Bewertung: -5.65
	Aktion: (3, 2) Bewertung: -9.12
	Aktion: (3, 3) Bewertung: -35.27
	Aktion: (3, 4) Bewertung: -32.47
	Aktion: (4, 1) Bewertung: -3.94
	Aktion: (4, 2) Bewertung: -14.72
	Aktion: (4, 3) Bewertung: -14.77
	Aktion: (4, 4) Bewertung: -21.86
	Aktion: (4, 5) Bewertung: -23.81
AI's Zug
AI nahm 1 von Stapel 4.

Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 3
Stapel 3: 4
Stapel 4: 4

Q-Values
	Aktion: (1, 1) Bewertung: -10.17
	Aktion: (1, 2) Bewertung: -22.08
	Aktion: (2, 1) Bewertung: 38.74
	Aktion: (2, 2) Bewertung: -0.07
	Aktion: (2, 3) Bewertung: -12.56
	Aktion: (3, 1) Bewertung: -9.86
	Aktion: (3, 2) Bewertung: -14.2
	Aktion: (3, 3) Bewertung: -26.46
	Aktion: (3, 4) Bewertung: -26.95
	Aktion: (4, 1) Bewertung: -8

Wähle Stapel:  2
Wähle Anzahl:  1



Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 2
Stapel 3: 4
Stapel 4: 4

Q-Values
	Aktion: (1, 1) Bewertung: -11.02
	Aktion: (1, 2) Bewertung: -18.78
	Aktion: (2, 1) Bewertung: -16.49
	Aktion: (2, 2) Bewertung: -20.39
	Aktion: (3, 1) Bewertung: -15.95
	Aktion: (3, 2) Bewertung: -22.45
	Aktion: (3, 3) Bewertung: -38.73
	Aktion: (3, 4) Bewertung: -35.29
	Aktion: (4, 1) Bewertung: -10.71
	Aktion: (4, 2) Bewertung: -17.34
	Aktion: (4, 3) Bewertung: -26.65
	Aktion: (4, 4) Bewertung: -42.12
AI's Zug
AI nahm 1 von Stapel 4.

Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 2
Stapel 3: 4
Stapel 4: 3

Q-Values
	Aktion: (1, 1) Bewertung: -27.86
	Aktion: (1, 2) Bewertung: -36.0
	Aktion: (2, 1) Bewertung: -31.26
	Aktion: (2, 2) Bewertung: -38.91
	Aktion: (3, 1) Bewertung: 44.14
	Aktion: (3, 2) Bewertung: -23.68
	Aktion: (3, 3) Bewertung: -27.1
	Aktion: (3, 4) Bewertung: -31.83
	Aktion: (4, 1) Bewertung: -20.01
	Aktion: (4, 2) Bewertung: -31.94
	Aktion: (4, 3) Bewertung: -40.47
Dein Zug


...


Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 2
Stapel 3: 1
Stapel 4: 1

Q-Values
	Aktion: (1, 1) Bewertung: -60.93
	Aktion: (1, 2) Bewertung: -62.89
	Aktion: (2, 1) Bewertung: -63.62
	Aktion: (2, 2) Bewertung: -62.56
	Aktion: (3, 1) Bewertung: -54.53
	Aktion: (4, 1) Bewertung: -59.58
AI's Zug
AI nahm 1 von Stapel 3.

Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 2
Stapel 3: 0
Stapel 4: 1

Q-Values
	Aktion: (1, 1) Bewertung: -51.32
	Aktion: (1, 2) Bewertung: -68.44
	Aktion: (2, 1) Bewertung: -62.06
	Aktion: (2, 2) Bewertung: -45.71
	Aktion: (4, 1) Bewertung: 80.0
Dein Zug


Wähle Stapel:  4
Wähle Anzahl:  1



Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 2
Stapel 3: 0
Stapel 4: 0

Q-Values
	Aktion: (1, 1) Bewertung: -75.41
	Aktion: (1, 2) Bewertung: -79.81
	Aktion: (2, 1) Bewertung: -79.73
	Aktion: (2, 2) Bewertung: -68.2
AI's Zug
AI nahm 2 von Stapel 2.

Spielfeld:
Stapel 0: 0
Stapel 1: 2
Stapel 2: 0
Stapel 3: 0
Stapel 4: 0

Q-Values
	Aktion: (1, 1) Bewertung: 100.0
	Aktion: (1, 2) Bewertung: -100.0
Dein Zug


Wähle Stapel:  1
Wähle Anzahl:  1



Spielfeld:
Stapel 0: 0
Stapel 1: 1
Stapel 2: 0
Stapel 3: 0
Stapel 4: 0

Q-Values
	Aktion: (1, 1) Bewertung: -100.0
AI's Zug
AI nahm 1 von Stapel 1.

GAME OVER
Mensch hat gewonnen


## Bewertung
Wie schlägt sich die AI schließlich: Gewinnt sie gegen einen Menschen? Gewinnt sie gegen die Gewinnstrategie nach Scam Nation?

Zunächst wurde die AI mit purer Exploration trainiert, was zur Folge hatte, dass sie die Pole Positions nicht verwendet hat. Sie war in etwa so gut, wie es ein Mensch ohne Gewinnstrategie auch wäre. Zum Ende des Spiels waren die Entscheidungen der AI hochwertig und zielsicher, zu Beginn eher weniger.

Mit der Einführung der Epsilon-Exploration wurde die AI regelrecht zum Nim-Endgegner. Gnadenlos zwingt sie ihre Kontrahenten in ungewinnbare Spielsituationen. Dabei verfolgt sie exakt die bekannte Gewinnstrategie mit den Pole Positions. Wenn die AI zuerst zieht, nimmt sie immer nur ein Objekt, was dem Gegenspieler die Pole Positions verwehrt. Falls der Mensch zuerst zieht und somit im dritten Zug die Pole Positions für sich gewinnt, macht die AI extra *kleine* Züge, in denen sie nur ein Objekt nimmt, was das Spiel maximal lang macht. Sie maximiert damit die Anzahl der Züge, die der Gegenspieler machen muss, welche stets ein (geringes) Risiko für Fehler bieten. Das passiert entweder unabsichtlich und entsteht durch die Sortierung der möglichen Aktionen im Programmcode, von denen die AI bei gleicher Bewertung einfach die erste nimmt oder durch zufällige Exploration des Gegenspielers beim Training: Wenn im Training der Computer gegen sich selbst spielt, kann es in 10% (Epsilonfaktor) der Fälle dazu kommen, dass ein Spieler, der die Siegerstellung der Pole Positions besitzt, eine zufällige Aktion durchführt, was ihn den Sieg kosten kann. Wie bereits im Kapitel der Spielregeln beschrieben, bietet genau das auch beim Spiel gegen/zwischen Menschen stets eine Siegeschance durch menschliches Versagen. An dieser Stelle bildet der Q-Learning Algorithmus mit Exploration und Exploitation also bemerkenswert detailgetreu die Realität ab.

Für das Standardspiel [3 5 7] ist die Gewinnstrategie für den Menschen nun klar und je nach Zug-Reihenfolge kann ein Mensch die AI besiegen. Beim Spiel mit anderen Spielfeldkonfigurationen (zum Beispiel [1 2 3 4 5] oder [2 6 7 8]), greift die bekannte Strategie jedoch nicht mehr unbedingt und hier zeigt die AI ihr wahres Potential. Sie ist in der Lage für jede erdenkliche Spielfeld-Variante eine Strategie zu entwickeln und hängt den Menschen siegessicher ab. Es lässt sich erkennen, dass sie dabei das Problem auf die bekannten Pole Positions reduziert. Diese Strategie könnte ein Mensch bei einer neuen Spielfeldkonfiguration ebenfalls verfolgen, nur ist der Weg zu den bekannten Schlüsselpositionen für den Menschen nicht mehr so offensichtlich wie zuvor. Die Strategie der AI hingegen umfasst bereits weitere (neue) Pole Positions.

## Herausforderungen
Die Datenstruktur der Q-Table und der Zugriff auf dessen Einträge über die Zustandskodierung als Arrays, erweiste sich als überaus schwer begreiflich. Diese Form wurde im vorangehenden Unterricht nicht behandelt und musste *explorativ* implementiert werden, anstatt, dass bereits behandelte Datenstrukturen *exploitet* werden konnten.

Die immense positive Wirkung der Epsilon-Exploration auf die Fähigkeit der AI ist nicht offensichtlich gewesen. Das etwas zufällig wirkende Verhalten der AI zu Beginn der Spiele wurde beinahe als Endresultat akzeptiert.

## Zusammenfassung
Mithilfe des Q-Learning Algorithmus konnte eine AI entwickelt werden, die das Nim-Spiel mit beliebiger Spielfeldkonfiguration **beherrscht**. Dabei konnte eine sinnvolle und **flexible** Zustandskodierung mit Arrays durchgesetzt werden.

Dem menschlichen Spieler ist es möglich, eine Spielfeldkonfiguration vorzugeben, auf der eine AI trainiert wird, die analog zu der *Gewinnstrategie nach Scam Nation* agiert. Beim Spielen gegen die AI kann der Mensch seine Fähigkeiten auf die Probe stellen oder auf die Informationen der Q-Table zurückgreifen. Durch einen Blick auf die Q-Values der möglichen Aktionen kann oft schon nach wenigen Zügen festgestellt werden, dass das Spiel bereits entschieden ist. Ein Mensch kann kaum darauf hoffen, dass die AI in einer echten Partie ihre Siegerstellung durch einen Fehler aufgibt, wie es womöglich ein Mensch mit jedem Zug riskiert.

Als abschließende Note lässt sich für diesen Fall festhalten:
**Exploitation ist wichtig. Lernt die AI *random*, spielt sie auch eher *random*.**